In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import FactorAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
data1 = pd.read_csv(r"datasets\imputed_data_QS_RF.csv")
data2 = pd.read_csv(r"datasets\imputed_data_QS_XGB.csv")
data2_sub = data2[['Institution Name', 'Year', 'IRN_SCORE', 'EO_SCORE', 'S_SCORE']]

In [3]:
# 进行合并
merged_df = pd.merge(data1, data2_sub, on=['Institution Name', 'Year'], how='left')
# 计算平均值并创建新列
merged_df['IRN_SCORE'] = (merged_df['IRN_SCORE_x'] + merged_df['IRN_SCORE_y']) / 2
merged_df['EO_SCORE']  = (merged_df['EO_SCORE_x']  + merged_df['EO_SCORE_y'])  / 2
merged_df['S_SCORE']   = (merged_df['S_SCORE_x']   + merged_df['S_SCORE_y'])   / 2
merged_df.drop(columns=['IRN_SCORE_x', 'IRN_SCORE_y',
                        'EO_SCORE_x',  'EO_SCORE_y',
                        'S_SCORE_x',   'S_SCORE_y'], inplace=True)
data = merged_df

In [4]:
numeric_data = data.drop(columns=['Rank', 'Institution Name', 'Country', 'Year','CS','CF','RI','Age','STA','Overall'])
numeric_data_standardized = (numeric_data - numeric_data.mean()) / numeric_data.std()
numeric_data_standardized.head()

,AR_SCORE,ER_SCORE,FS_SCORE,CPF_SCORE,IF_SCORE,IS_SCORE,IRN_SCORE,EO_SCORE,S_SCORE
0,2.860526,2.722843,2.177381,2.302591,1.800375,1.988527,1.232425,2.918425,1.913042
1,2.860526,2.722843,2.177381,2.285786,1.789078,1.256705,1.574039,2.961351,1.930189
2,2.860526,2.722843,2.120115,2.302591,1.701525,1.334891,1.549857,2.940762,1.956016
3,2.840967,2.177224,2.177381,2.305952,1.613973,1.772733,0.580724,1.317907,1.042702
4,2.860526,2.722843,2.177381,1.576632,1.726944,2.038566,1.583625,2.950500,1.603607


In [5]:
file_patht = r"datasets/cleaned_data_THE.csv"
dft = pd.read_csv(file_patht, encoding='latin1')

In [6]:
# Check missing
missing = dft[dft.columns].isnull().sum().values / dft.shape[0]   # Calculate missing values ratio
for i,column in zip(missing, dft.columns):
    print(f"{i * 100:.2f}% missing in",column)

0.00% missing in Rank
0.00% missing in Name
0.00% missing in Country
0.00% missing in Year
0.00% missing in Student Population
0.00% missing in Students to Staff Ratio
0.00% missing in International Students
0.00% missing in Teaching
0.00% missing in Research Environment
0.00% missing in Research Quality
0.00% missing in Industry Impact
0.00% missing in International Outlook
0.00% missing in Overall


In [7]:
df_qs = data.copy()
dft

,Rank,Name,Country,Year,Student Population,Students to Staff Ratio,International Students,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Overall
0,1,California Institute of Technology,United States,2016,2243,6.9,26,95.6,97.6,99.8,97.8,64.0,95.2000
1,2,University of Oxford,United Kingdom,2016,19920,11.6,34,86.5,98.9,98.8,73.1,94.4,94.2000
2,3,Stanford University,United States,2016,15596,7.8,22,92.5,96.2,99.9,63.3,76.3,93.9000
3,4,University of Cambridge,United Kingdom,2016,18810,11.8,34,88.2,96.7,97.0,55.0,91.5,92.8000
4,5,Massachusetts Institute of Technology,United States,2016,11074,9.0,33,89.4,88.6,99.7,95.4,84.0,92.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14497,2088,Technological University of Pereira,Colombia,2025,17326,17.1,0,15.5,2.8,10.2,18.5,33.3,11.6820
14498,2089,Universidad Pedagogica y Tecnologica de Colomb...,Colombia,2025,35767,17.6,0,14.2,7.7,5.0,16.1,34.0,11.1160
14499,2090,Instituto TecnolÂ¨Â®gico Metropolitano,Colombia,2025,24443,20.3,0,12.4,7.6,7.6,18.4,29.7,11.1055
14500,2091,Tula State University,Russian Federation,2025,18622,19.3,3,15.2,9.2,5.1,17.8,19.6,10.8640


In [8]:
# Check missing
missing2 = df_qs[df_qs.columns].isnull().sum().values / df_qs.shape[0]   # Calculate missing values ratio
for i,column in zip(missing2, df_qs.columns):
    print(f"{i * 100:.2f}% missing in",column)

0.00% missing in Rank
0.00% missing in Institution Name
0.00% missing in Country
0.00% missing in Year
0.00% missing in CS
0.00% missing in CF
0.00% missing in RI
0.00% missing in Age
0.00% missing in STA
0.00% missing in AR_SCORE
0.00% missing in ER_SCORE
0.00% missing in FS_SCORE
0.00% missing in CPF_SCORE
0.00% missing in IF_SCORE
0.00% missing in IS_SCORE
0.00% missing in Overall
0.00% missing in IRN_SCORE
0.00% missing in EO_SCORE
0.00% missing in S_SCORE


In [9]:
# 将非数字Rank处理成NaN，再转为浮点型
df_qs['Rank'] = pd.to_numeric(df_qs['Rank'], errors='coerce')
dft['Rank'] = pd.to_numeric(dft['Rank'], errors='coerce')

# 筛选出排名小于等于500的部分
qs_top500 = df_qs[df_qs['Rank'] <= 500].copy()
times_top500 = dft[dft['Rank'] <= 500].copy()
 
# 名称统一小写
df_qs['Institution Name'] = df_qs['Institution Name'].str.lower()
dft['Name'] = dft['Name'].str.lower()


In [10]:
qs_uni_names = df_qs['Institution Name'].dropna().unique()
times_uni_names = dft['Name'].dropna().unique()
 

In [11]:
# QS 名单中唯一学校数
qs_count = len(qs_uni_names)
print(f"QS 名单中唯一学校数量: {qs_count}")

# Times 名单中唯一学校数
times_count = len(times_uni_names)
print(f"Times 名单中唯一学校数量: {times_count}")


QS 名单中唯一学校数量: 2219
Times 名单中唯一学校数量: 2333


In [12]:
 
A_names = times_uni_names  # 比如 Times 名单
B_names = qs_uni_names     # 比如 QS 名单

threshold = 85
top_n = 3  # 每个 A 名称匹配 B 中前 N 个

# 存储结果
multi_to_multi_matches = []

for a in A_names:
    matches = process.extract(a, B_names, scorer=fuzz.ratio, limit=top_n)
    for b, score in matches:
        if score >= threshold:
            multi_to_multi_matches.append({
                "Times Name": a,
                "Matched QS Name": b,
                "Score": score
            })

# 构建 DataFrame
multi2multi_df = pd.DataFrame(multi_to_multi_matches)

# 排序查看
multi2multi_df = multi2multi_df.sort_values(by="Score", ascending=False)

multi2multi_df.head(10)


,Times Name,Matched QS Name,Score
1049,amirkabir university of technology,amirkabir university of technology,100
1135,kanazawa university,kanazawa university,100
1149,bauman moscow state technical university,bauman moscow state technical university,100
1146,university of regina,university of regina,100
1144,auburn university,auburn university,100
1142,manchester metropolitan university,manchester metropolitan university,100
1140,south china university of technology,south china university of technology,100
1139,united arab emirates university,united arab emirates university,100
2730,soonchunhyang university,soonchunhyang university,100
1136,university of salamanca,university of salamanca,100


In [13]:
multi2multi_df

,Times Name,Matched QS Name,Score
1049,amirkabir university of technology,amirkabir university of technology,100
1135,kanazawa university,kanazawa university,100
1149,bauman moscow state technical university,bauman moscow state technical university,100
1146,university of regina,university of regina,100
1144,auburn university,auburn university,100
...,...,...,...
625,kazan federal university,ural federal university,85
1480,the university of jordan,the university of osaka,85
632,university of plymouth,university of utah,85
2997,university of rajshahi,university of hail,85


In [14]:
 
threshold = 85
top_n = 3  # 每个 B 名称匹配 A 中前 N 个

# 存储结果
reverse2_matches = []

for b in B_names:
    matches = process.extract(b, A_names, scorer=fuzz.ratio, limit=top_n)
    for a, score in matches:
        if score >= threshold:
            reverse2_matches.append({
                "Matched QS Name": b,
                "Times Name": a,
                "Score": score
            })

# 构建 DataFrame
reverse_multi_df2 = pd.DataFrame(reverse2_matches)

# 排序查看
reverse_multi_df2 = reverse_multi_df2.sort_values(by="Score", ascending=False).reset_index(drop=True)

# 展示前 10 行
print(reverse_multi_df2.head(10))


                   Matched QS Name                       Times Name  Score
0             yamaguchi university             yamaguchi university    100
1      edinburgh napier university      edinburgh napier university    100
2  beijing institute of technology  beijing institute of technology    100
3    islamic university of madinah    islamic university of madinah    100
4    harbin engineering university    harbin engineering university    100
5               khalifa university               khalifa university    100
6            hacettepe university              hacettepe university    100
7          george mason university          george mason university    100
8         sultan qaboos university         sultan qaboos university    100
9                tulane university                tulane university    100


In [15]:

def clean_name(name):
    """去掉 the、university、of 等停用词"""
    stop_words = {"the", "university", "of"}
    words = name.lower().split()
    return ' '.join(word for word in words if word not in stop_words)

# 建立原始 -> 清洗后映射
qs_name_map = {name: clean_name(name) for name in qs_uni_names}
times_name_map = {name: clean_name(name) for name in times_uni_names}

# 开始模糊匹配
threshold = 85
matched = []

for orig_name_t, cleaned_name_t in times_name_map.items():
    result = process.extractOne(cleaned_name_t, list(qs_name_map.values()), scorer=fuzz.ratio)
    if result:
        matched_clean_qs, score = result[0], result[1]
        if score >= threshold:
            # 找回原始 QS 名称
            matched_orig_qs = next(k for k, v in qs_name_map.items() if v == matched_clean_qs)
            matched.append((orig_name_t, matched_orig_qs, score))

# 构建结果 DataFrame
matched_df = pd.DataFrame(matched, columns=['Times Name', 'Matched QS Name', 'Score'])


In [16]:
 
merged_matches = pd.concat([multi2multi_df, matched_df], ignore_index=True)
merged_matches2 = pd.concat([reverse_multi_df2, merged_matches], ignore_index=True)
merged_matches3 = merged_matches2.drop_duplicates(subset=["Times Name", "Matched QS Name"])
merged_matches3 = merged_matches3.sort_values(by="Score", ascending=False).reset_index(drop=True)
merged_matches3


,Matched QS Name,Times Name,Score
0,yamaguchi university,yamaguchi university,100
1,naresuan university,naresuan university,100
2,king mongkut's university of technology north ...,king mongkut's university of technology north ...,100
3,cyprus university of technology,cyprus university of technology,100
4,korea university,korea university,100
...,...,...,...
3859,kangwon national university,pusan national university,85
3860,university of maynooth,university of bath,85
3861,university of nizwa,university of arizona,85
3862,the university of northampton,university of southampton,85


In [17]:
from fuzzywuzzy import process, fuzz

# 找出已经匹配成功的 Times Name
matched_times_names = set(merged_matches3['Times Name'])

# 找出未匹配成功的 Times Name（原始的，不是clean_name后的）
unmatched_times_names = [name for name in times_uni_names if name not in matched_times_names]

# 建立原始 QS 名称列表（不做清洗）
qs_names_raw = list(qs_uni_names)

# 重新进行匹配（不清洗）
rematched = []
threshold_rematch = 85  # 可以设置稍高的阈值

for name in unmatched_times_names:
    result = process.extractOne(name, qs_names_raw, scorer=fuzz.ratio)
    if result:
        matched_qs, score = result
        if score >= threshold_rematch:
            rematched.append((name, matched_qs, score))

# 构建 rematched_df
rematched_df = pd.DataFrame(rematched, columns=['Times Name', 'Matched QS Name', 'Score'])
 

In [18]:
rematched_df = rematched_df.apply(lambda col: col.str.lower() if col.dtypes == 'object' else col)
merged_matches4= pd.concat([rematched_df, merged_matches3], ignore_index=True)
merged_matches4 = merged_matches4.drop_duplicates(subset=["Times Name", "Matched QS Name"])
merged_matches4

,Times Name,Matched QS Name,Score
0,yamaguchi university,yamaguchi university,100
1,naresuan university,naresuan university,100
2,king mongkut's university of technology north ...,king mongkut's university of technology north ...,100
3,cyprus university of technology,cyprus university of technology,100
4,korea university,korea university,100
...,...,...,...
3859,pusan national university,kangwon national university,85
3860,university of bath,university of maynooth,85
3861,university of arizona,university of nizwa,85
3862,university of southampton,the university of northampton,85


In [19]:
merged_matches4.to_csv('datasets/matched_final_lower83.csv', index=False)

In [21]:
fuz = pd.read_csv(r"datasets/matched_final_.csv")

In [22]:
data = [
    ["university of pittsburgh–pittsburgh campus", "university of pittsburgh", 53],
    ["university of são paulo", "universidade de são paulo", 53],
    ["university of colorado denver/anschutz medical campus", "university of colorado, denver", 55],
    ["university of mannheim", "universität mannheim", 57],
    ["university of konstanz", "universität konstanz", 57],
    ["university of stuttgart", "universität stuttgart", 60],
    ["university of innsbruck", "universität innsbruck", 60],
    ["university of hohenheim", "universität hohenheim", 60],
    ["ohio state university (main campus)", "ohio state university", 61],
    ["university of strasbourg", "université de strasbourg", 61],
    ["university of new mexico (main campus)", "university of new mexico", 61],
    ["national research nuclear university mephi", "national research nuclear university mephi (moscow engineering physics institute)", 62],
    ["pierre and marie curie university", "université pierre et marie curie (upmc)", 63],
    ["goethe university frankfurt", "goethe-university frankfurt am main", 63],
    ["montpellier university", "université de montpellier", 63],
    ["university of erlangen–nuremberg", "universität erlangen-nürnberg", 64],
    ["ruhr university bochum", "ruhr-universität bochum", 65],
    ["renmin university of china", "renmin (people’s) university of china", 65],
    ["humboldt university of berlin", "humboldt-universität zu berlin", 67],
    ["pompeu fabra university", "universitat pompeu fabra", 67],
    ["charles darwin university", "charles university", 67],
    ["university of oregon", "oregon state university", 67],
    ["paris–sorbonne university – paris 4", "université paris-sorbonne (paris iv)", 72],
    ["paris descartes university", "université paris descartes", 73],
    ["university of padua", "university of padova", 73],
    ["pontifical javeriana university", "pontificia universidad javeriana", 73],
    ["university of georgia (usa)", "the university of georgia", 74],
    ["eth zurich", "university of zurich", 75],
    ["claude bernard university lyon 1", "université claude bernard lyon 1", 75],
    ["new york university", "new york university (nyu)", 76],
    ["vita-salute san raffaele university", "università vita-salute san raffaele", 77],
    ["paris didérot university – paris 7", "université paris didérot - paris 7", 78],
    ["university of st gallen", "university of st gallen (hsg)", 78],
    ["arizona state university (tempe)", "arizona state university", 79],
    ["justus liebig university giessen", "justus-liebig-universität giessen", 79],
    ["sungkyunkwan university (skku)", "sungkyunkwan university", 80],
    ["university at buffalo", "university at buffalo suny", 80],
    ["johannes gutenberg university of mainz", "johannes gutenberg universität mainz", 80],
    ["universität heidelberg", "universität freiburg", 81],  # 这里请你确认是否匹配错了（原图中是heidelberg vs freiburg）
    ["university of east anglia", "university of east anglia (uea)", 81],
    ["queen’s university", "queen's university", 82],
    ["central queensland university", "central queensland university (cquniversity australia)", 71],
    ["university of virginia (main campus)", "university of virginia", 77],
    ["university of valencia", "universitat de valencia", 84],
    ["technical university of berlin", "technische universität berlin", 85],
    ["university of bremen", "universität bremen", 89],
    ["free university of berlin", "freie universität berlin", 90],
    ["university of bayreuth", "universität bayreuth", 90],
    ["university of freiburg", "universität freiburg", 90],
    ["university of hamburg", "universität hamburg", 90],
    ["university of potsdam", "universität potsdam", 90],
    ["northeastern university, us", "northeastern university", 92],
    ["jordan university of science and technology", "jordan university of science & technology", 95],
    ["jordan university of science and technology", "jordan university of science & technology", 93],
    ["unsw sydney", "the university of new south wales (unsw sydney)", 56],
    ["unsw sydney", "the university of new south wales ", 56], 
    ["justus liebig university giessen", "justus-liebig-university giessen", 100],
    ["soochow university, taiwan", "soochow university (taiwan)", 100],
    ["government college university faisalabad", "government college university, faisalabad", 99],
    ["sungkyunkwan university (skku)", "sungkyunkwan university(skku)", 98],
    ["asia university, taiwan", "asia university taiwan", 98],
    ["university of hai'l", "university of hail", 97],
    ["university of nice sophia antipolis", "universit? nice sophia-antipolis", 96],
    ["johannes gutenberg university of mainz", "johannes gutenberg universit?t mainz", 95],
    ["university of wroc?aw", "university of wroclaw", 95],
    ["kanagawa university", "kanazawa university", 95],
    ["paris lodron universit?t salzburg", "paris lodron university of salzburg", 94],
    ["university of california, berkeley", "university of california, berkeley (ucb)", 93],
    ["national university of singapore", "national university of singapore (nus)", 93],
    ["university of california, santa barbara", "university of california, santa barbara (ucsb)", 93],
    ["university of duisburg-essen", "universit?t duisburg-essen", 93],
    ["ruhr university bochum", "ruhr-universit?t bochum", 93],
    ["y?ld?z technical university", "yildiz technical university", 93],
    ["university of northern british columbia (unbc)", "university of northern british columbia", 93],
    ["international islamic university, islamabad", "international islamic university islamabad (iiu)", 93],
    ["university of north florida", "university of south florida", 93],
    ["university of new brunswick unb", "university of new brunswick", 93],
    ["university of colima", "university of coimbra", 93],
    ["poznan芒莽芒陆 university of life sciences", "poznan university of life sciences", 93],
    ["university of california, los angeles", "university of california, los angeles (ucla)", 92],
    ["university of california, san diego", "university of california, san diego (ucsd)", 92],
    ["northeastern university, us", "northeastern university", 92],
    ["university of illinois chicago", "university of illinois chicago (uic)", 92],
    ["queen芒芒芒's university", "queen's university", 92],
    ["american university of beirut", "american university of beirut (aub)", 92],
    ["tomas bata university in zl芒莽芒陆n", "tomas bata university in zlin", 92],
    ["university of south alabama", "university of south alabama (usa)", 92],
    ["lovely professional university", "lovely professional university (lpu)", 92],
    ["kyungpook national university (knu)", "kyungpook national university", 92],
    ["university of east anglia", "university of east anglia (uea)", 91],
    ["university of stuttgart", "universit?t stuttgart", 91],
    ["king abdulaziz university", "king abdulaziz university (kau)", 91],
    ["university of innsbruck", "universit?t innsbruck", 91],
    ["australian national university (anu)", "australian national university",90],
    ["australian national university", "australian national university (anu)",90],
    ["shahid beheshti university", "shahid beheshti university (sbu)", 91],
    ["university rey juan carlos", "universidad rey juan carlos", 91],
    ["czech university of life sciences prague (czu)", "czech university of life sciences in prague", 91],
    ["university of regensburg", "universit?t regensburg", 91],
    ["prince sultan university (psu)", "prince sultan university", 91],
    ["free university of berlin", "freie universit?t berlin", 90],
    ["university of mannheim", "universit?t mannheim", 90],
    ["university of konstanz", "universit?t konstanz", 90],
    ["university of st gallen", "university of st gallen (hsg)", 90],
    ["china university of geosciences, wuhan", "china university of geosciences", 90],
    ["university of ghana", "university of granada", 90],
    ["university of hamburg", "universit?t hamburg", 90],
    ["university of potsdam", "universit?t potsdam", 90],
    ["seoul national university of science and technology (seoultech)", "seoul national university of science and technology", 90],
    ["louisiana tech university", "louisiana state university", 90],
    ["szent istv芒莽芒芒n university", "szent istvan university", 90],
    ["american international university 芒莽c bangladesh", "american international university-bangladesh (aiub)", 90],
    ["university of massachusetts", "university of massachusetts boston", 89],
    ["university at buffalo", "university at buffalo suny", 89],
    ["university of south florida (tampa)", "university of south florida", 89],
    ["university of strasbourg", "universit?de strasbourg", 89],
    ["multimedia university", "multimedia university (mmu)", 89],
    ["the islamia university of bahawalpur", "the islamia university of bahawalpur,pakistan", 89],
    ["warsaw university of life sciences 芒莽c sggw", "warsaw university of life sciences 芒莽c sggw (wuls-sggw)", 89],
    ["symbiosis international university", "symbiosis international (deemed university)", 89],
    ["imam mohammad ibn saud islamic university", "imam mohammad ibn saud islamic university 芒莽c imsiu", 89],
    ["pontificia universidad cat芒莽芒卢lica del ecuador", "pontificia universidad cat?lica del per?", 89],
    ["california institute of technology", "california institute of technology (caltech)", 88],
    ["new york university", "new york university (nyu)", 88],
    ["humboldt university of berlin", "humboldt-universit?t zu berlin", 88],
    ["芒莽芒莽cole normale sup芒莽芒莽rieure", "芒莽芒莽cole normale sup芒莽芒莽rieure de lyon", 88],
    ["芒莽芒莽cole polytechnique", "ecole polytechnique", 88],
    ["izmir institute of technology", "izmir institute of technology (iztech)", 88],
    ["nanjing tech university", "nanjing university", 88],
    ["istanbul bilgi university", "istanbul bilgi ?niversitesi", 88],
    ["the university of queensland","the university of queensland (uq)",90],
    ["the university of queensland (uq)","the university of queensland",90],
    ["the University of Western Australia (uwa)","the University of Western Australia",99],
    ["the University of Western Australia","the University of Western Australia (uwa)",99],
    ["university of technology sydney","university of technology sydney (uts)",99],
    ["university of technology sydney (uts)","university of technology sydney",99], 
    ["unsw sydney", "the university of new south wales (unsw)", 56],
] 
matched_df_extra = pd.DataFrame(data, columns=["Times Name", "Matched QS Name", "Score"])

In [23]:
# 假设 matched_df2 是之前的匹配结果
rematched_fuz = pd.concat([fuz, matched_df_extra], ignore_index=True)
# 去重以防重复（可选）
rematched_fuz = rematched_fuz.drop_duplicates(subset=["Times Name", "Matched QS Name"])

In [24]:
# 明确列出要删除的“Times Name” + “Matched QS Name”组合
to_remove = [
    ("east china university of science and technology", "macau university of science and technology"),
    ("école polytechnique", "ecole polytechnique"),
    ("jordan university of science and technology", "iran university of science and technology"),
    ("national taichung university of science and technology", "national taiwan university of science and technology"),
    ("ufa university of science and technology", "agh university of science and technology"),
    ("chiba institute of technology", "shibaura institute of technology"),
    ("university of west london", "university of east london"),
    ("technische universit?t ilmenau", "technische universität wien"),
    ("sahand university of technology", "isfahan university of technology"),
    ("national university of sciences and technology", "the national university of science and technology misis"),
    ("kunming university of science and technology", "nanjing university of science and technology"),
    ("indian institute of technology mandi", "indian institute of technology indore"),
    ("kohat university of science and technology", "khalifa university of science and technology"),
    ("mirpur university of science and technology", "missouri university of science and technology"),
    ("universitas sriwijaya", "universitas brawijaya"),
]

# 从 matched_df2 中剔除这些行
rematched_fuz = rematched_fuz[~rematched_fuz[["Times Name", "Matched QS Name"]].apply(tuple, axis=1).isin(to_remove)]


In [25]:
rematched_fuz

,Times Name,Matched QS Name,Score
0,yamaguchi university,yamaguchi university,100
1,universitas muhammadiyah yogyakarta,universitas muhammadiyah yogyakarta,100
2,university of waterloo,university of waterloo,100
3,university of alberta,university of alberta,100
4,universidade federal do rio grande do sul,universidade federal do rio grande do sul,100
...,...,...,...
1536,the university of queensland (uq),the university of queensland,90
1537,the University of Western Australia (uwa),the University of Western Australia,99
1538,the University of Western Australia,the University of Western Australia (uwa),99
1540,university of technology sydney (uts),university of technology sydney,99


In [26]:
# Step 1: 准备模糊匹配映射表
mapping_df = rematched_fuz[['Times Name', 'Matched QS Name']].copy()
mapping_df.columns = ['Times Name', 'QS Name']

# Step 2: 将 Times 数据和映射表连接
dft_mapped = dft.merge(mapping_df, left_on='Name', right_on='Times Name', how='inner')

# Step 3: 将 QS 数据和 Times-映射数据再合并
merged = dft_mapped.merge(
    df_qs, 
    left_on=['QS Name', 'Year'], 
    right_on=['Institution Name', 'Year'], 
    how='inner',
    suffixes=('_Times', '_QS')
)

# Step 4: 可选统一标准名（如保留 QS 名称）
#merged['Standard Name'] = merged['QS Name']


In [27]:
merged

,Rank_Times,Name,Country_Times,Year,Student Population,Students to Staff Ratio,International Students,Teaching,Research Environment,Research Quality,...,AR_SCORE,ER_SCORE,FS_SCORE,CPF_SCORE,IF_SCORE,IS_SCORE,Overall_QS,IRN_SCORE,EO_SCORE,S_SCORE
0,1,university of oxford,United Kingdom,2018,20410,11.2,38,86.7,99.5,99.1,...,100.0,100.0,100.0,76.3,98.600000,98.500000,95.300000,94.973568,97.390690,85.563685
1,2,university of cambridge,United Kingdom,2018,18390,10.9,35,87.8,97.8,97.5,...,100.0,100.0,100.0,78.3,97.400000,97.700000,95.600000,94.883883,99.254512,85.962091
2,3,stanford university,United States,2018,15845,7.5,22,89.1,96.7,99.9,...,100.0,100.0,100.0,99.4,99.600000,72.700000,98.700000,94.601807,99.517791,95.491991
3,4,california institute of technology,United States,2018,2209,6.5,27,90.3,97.5,99.5,...,99.5,85.4,100.0,100.0,93.400000,89.200000,97.700000,65.373699,59.643697,69.594482
4,5,massachusetts institute of technology,United States,2018,11177,8.7,34,87.3,91.9,100.0,...,100.0,100.0,100.0,99.9,100.000000,96.100000,100.000000,84.549896,98.476302,94.991632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,2065,universidad peruana de ciencias aplicadas (upc),Peru,2025,79091,28.0,1,11.9,8.7,13.9,...,11.3,22.6,3.0,1.1,13.600000,1.500000,31.012500,8.000000,3.900000,1.100000
5273,2073,universidad michoacana de san nicolâ¨â¢s de hi...,Mexico,2025,43423,24.7,1,14.4,8.6,11.1,...,4.1,1.8,2.5,2.0,12.046479,3.971429,49.572727,14.000000,1.500000,1.000000
5274,2075,irkutsk state university,Russian Federation,2025,13152,13.9,5,18.0,10.2,5.7,...,3.3,1.8,14.1,1.3,1.900000,3.800000,32.836667,26.400000,5.200000,1.200000
5275,2077,universidad autâ¨â®noma del estado de hidalgo,Mexico,2025,38714,22.3,1,13.7,9.5,10.4,...,3.2,6.0,45.2,1.4,2.000000,1.600000,49.572727,8.300000,2.100000,5.800000


In [28]:
# rematched_fuz = rematched_fuz.rename(columns={'Times Name': 'Name'})
# rematched_fuz = rematched_fuz.drop(columns=['Score'])
# rematched_fuz
# #matched_df2.to_csv('datasets/matched_final_lower.csv', index=False)

In [29]:
# 找出 Name（Times）有多个对应 QS 的
duplicated_names = rematched_fuz.groupby('Times Name').filter(lambda x: len(x) > 1)
duplicated_names
# 找出 Matched QS Name 有多个对应 Times 的
duplicated_qs_names = rematched_fuz.groupby('Matched QS Name').filter(lambda x: len(x) > 1)

# 合并去重显示所有匹配不唯一的项
non_unique_matches = pd.concat([duplicated_names, duplicated_qs_names]).drop_duplicates()

# # 显示结果
# non_unique_matches.sort_values(by='Name').reset_index(drop=True)


In [30]:
non_unique_matches

,Times Name,Matched QS Name,Score
0,yamaguchi university,yamaguchi university,100
1,universitas muhammadiyah yogyakarta,universitas muhammadiyah yogyakarta,100
4,universidade federal do rio grande do sul,universidade federal do rio grande do sul,100
7,universidad peruana de ciencias aplicadas (upc),universidad peruana de ciencias aplicadas (upc),100
9,universidad del desarrollo,universidad del desarrollo,100
...,...,...,...
1516,american international university 芒莽c bangladesh,american international university-bangladesh (...,90
1526,pontificia universidad cat芒莽芒卢lica del ecuador,pontificia universidad cat?lica del per?,89
1533,nanjing tech university,nanjing university,88
1536,the university of queensland (uq),the university of queensland,90


In [31]:
# 1. 获取已匹配的名称集合
matched_qs_set = set(rematched_fuz['Matched QS Name'].str.lower().str.strip())
matched_times_set = set(rematched_fuz['Times Name'].str.lower().str.strip())

# 2. 原始所有大学集合
all_qs_set = set(df_qs['Institution Name'].str.lower().str.strip())
all_times_set = set(dft['Name'].str.lower().str.strip())

# 3. 计算未匹配的部分
unmatched_qs = sorted(all_qs_set - matched_qs_set)
unmatched_times = sorted(all_times_set - matched_times_set)

In [32]:
china_unmatched_qs = [name for name in unmatched_qs if "austra" in name]
print("❗unmatch QS include  ：")
for name in china_unmatched_qs:
    print(name)
china_unmatched_times = [name for name in unmatched_times if "austra" in name]
print("❗unmatch Times include  ：")
for name in china_unmatched_times:
    print(name)


❗unmatch QS include  ：
central queensland university australia (cquniversity)
the university of newcastle, australia (uon)
universidad austral
universidad austral - argentina
universidad austral de chile
university of new england australia
❗unmatch Times include  ：
austral university
austral university of chile
federation university australia


In [33]:
# dft['Name'] = dft['Name'].str.lower()
# dft_merged = pd.merge(dft, matched_df22, on='Name', how='left')
# dft_merged = dft_merged.rename(columns={'Matched QS Name': 'Institution Name'})
# dft_merged

In [34]:

# dft_merged['Institution Name'] = dft_merged['Institution Name'].str.lower().str.strip()
# df_qs['Institution Name'] = df_qs['Institution Name'].str.lower().str.strip()
# dft_merged['Year'] = dft_merged['Year'].astype(int)
# df_qs['Year'] = df_qs['Year'].astype(int)
 
# fin_merged2 = pd.merge(
#     dft_merged,
#     df_qs,
#     on=['Institution Name', 'Year'],
#     how='inner',
#     suffixes=('_th', '_qs')   
# )
 
# fin_merged2 

In [35]:
merged.to_csv('datasets/merged.csv', index=False)

In [36]:
print(merged.columns.tolist())

['Rank_Times', 'Name', 'Country_Times', 'Year', 'Student Population', 'Students to Staff Ratio', 'International Students', 'Teaching', 'Research Environment', 'Research Quality', 'Industry Impact', 'International Outlook', 'Overall_Times', 'Times Name', 'QS Name', 'Rank_QS', 'Institution Name', 'Country_QS', 'CS', 'CF', 'RI', 'Age', 'STA', 'AR_SCORE', 'ER_SCORE', 'FS_SCORE', 'CPF_SCORE', 'IF_SCORE', 'IS_SCORE', 'Overall_QS', 'IRN_SCORE', 'EO_SCORE', 'S_SCORE']


In [37]:
qs_top500['Institution Name'] = qs_top500['Institution Name'].str.lower()
times_top500['Name'] = times_top500['Name'].str.lower()
qs_names_set = set(qs_top500['Institution Name'].dropna().unique())
times_names_set = set(times_top500['Name'].dropna().unique())
matched_qs_names = set(rematched_fuz['Matched QS Name'].str.lower())
matched_times_names = set(rematched_fuz['Times Name'].str.lower())
unmatched_qs_names = qs_names_set - matched_qs_names
unmatched_times_names = times_names_set - matched_times_names
print(" QS num：", len(qs_names_set))
print(" Times num：", len(times_names_set))
print("unmarch QS num：", len(unmatched_qs_names))
print("unmarch Times num：", len(unmatched_times_names))

 QS num： 1003
 Times num： 752
unmarch QS num： 314
unmarch Times num： 158


In [ ]:
# fin_merged2[fin_merged2['Name'].str.contains('queensland', case=False, na=False)]